# U.S. Medical Insurance Costs : Regional Insurance Analysis Dashboard

In [29]:
import csv

region_counts = {}

with open("insurance.csv") as insurance_csv_file:
    table_to_analyse = csv.DictReader(insurance_csv_file)
    data = list(table_to_analyse)


def avg_charges_by_region(data):
    counts_by_region = {}
    total_charges_by_region = {}

    
    for person in data:
        region = person['region']
        charges = float(person['charges'])
        
        if region in total_charges_by_region:
            total_charges_by_region[region] += charges
        else :
            total_charges_by_region[region] = charges

        if region in counts_by_region:
            counts_by_region[region] += 1
        else:
            counts_by_region[region] = 1

    avg_by_region = {}
    for region in total_charges_by_region:
        avg_by_region[region] = total_charges_by_region[region] / counts_by_region[region]
    return avg_by_region

average_charges = avg_charges_by_region(data)
print("Average charges by region: ", average_charges)
        

Average charges by region:  {'southwest': 12346.93737729231, 'southeast': 14735.411437609895, 'northwest': 12417.575373969228, 'northeast': 13406.3845163858}


In [7]:
import pandas as pd

# Convert CSV to pandas DataFrame for easier analysis
with open("insurance.csv") as insurance_csv_file:
    data = pd.read_csv(insurance_csv_file)

# Calculate key metrics by region
def regional_analysis(data):
    # Group by region and calculate statistics
    regional_stats = data.groupby('region').agg({
        'charges': ['mean', 'median', 'std', 'min', 'max', 'count'],
        'age': ['mean', 'min', 'max'],
        'bmi': ['mean', 'median'],
        'smoker': lambda x: (x == 'yes').mean() * 100,  # Percentage of smokers
        'children': ['mean', 'sum']
    })
    
    # Flatten the column hierarchy
    regional_stats.columns = ['_'.join(col).strip() for col in regional_stats.columns.values]
    
    # Additional calculations
    # Calculate charge per BMI point by region - modified to avoid deprecation warning
    charge_per_bmi = data.groupby('region').apply(
        lambda x: (x['charges'] / x['bmi']).mean(), include_groups=False
    ).reset_index(name='charge_per_bmi_point')
    
    return regional_stats, charge_per_bmi

# Calculate regional statistics
regional_stats, charge_per_bmi = regional_analysis(data)

# Export results to CSV for Power BI import
regional_stats.reset_index().to_csv('regional_stats.csv', index=False)
charge_per_bmi.to_csv('charge_per_bmi.csv', index=False)

# Also export the full dataset for Power BI
data.to_csv('insurance_clean.csv', index=False)

print("Analysis complete and files exported for Power BI")

Analysis complete and files exported for Power BI


In [9]:
# Create age group categories for better analysis
data['age_group'] = pd.cut(data['age'], bins=[0, 18, 30, 45, 65, 100], 
                          labels=['0-18', '19-30', '31-45', '46-65', '65+'])

# Regional analysis by age group - add observed=False to maintain current behavior
regional_age_stats = data.groupby(['region', 'age_group'], observed=False).agg({
    'charges': 'mean',
    'bmi': 'mean',
    'smoker': lambda x: (x == 'yes').mean() * 100
}).reset_index()

regional_age_stats.to_csv('regional_age_stats.csv', index=False)

# Regional analysis by smoker status
smoker_stats = data.groupby(['region', 'smoker']).agg({
    'charges': ['mean', 'count'],
    'bmi': 'mean',
    'age': 'mean'
})

smoker_stats.columns = ['_'.join(col).strip() for col in smoker_stats.columns.values]
smoker_stats.reset_index().to_csv('regional_smoker_stats.csv', index=False)